In [5]:
import numpy as np

In [6]:
def single_pass_variance(arr):
    """
    Compute single-pass variance approximation for a given array.
    
    Parameters:
        arr (numpy.ndarray): Input array.
    
    Returns:
        tuple: A tuple containing:
            - approx_variance (float): Approximate variance of the array.
            - meta_data (dict): Metadata containing mean, count, and M2.
    """
    count = 0
    mean = 0.0
    M2 = 0.0

    for x in arr:
        count += 1
        delta = x - mean
        mean += delta / count
        delta2 = x - mean
        M2 += delta * delta2

    approx_variance = M2 / count if count > 1 else 0.0
    meta_data = {"mean": mean, "count": count, "M2": M2}

    return approx_variance, meta_data

In [25]:
def combine_variance2(meta1, meta2):
    """
    Combine two variance approximations using their metadata.
    Supports positive and negative counts.

    Parameters:
        meta1 (dict): Metadata of the first array.
        meta2 (dict): Metadata of the second array (count can be negative for removal).

    Returns:
        tuple: A tuple containing:
            - combined_variance (float): Combined variance of the two arrays.
            - combined_meta (dict): Metadata containing combined mean, count, and M2.
    """
    count1, mean1, M21 = meta1["count"], meta1["mean"], meta1["M2"]
    count2, mean2, M22 = meta2["count"], meta2["mean"], meta2["M2"]

    combined_count = count1 + count2
    delta = mean2 - mean1

    # Handle edge cases where the combined count is invalid
    if combined_count <= 0:
        # If the combined count is zero or negative, return zeroed metadata
        return 0.0, {"mean": 0.0, "count": 0, "M2": 0.0}

    combined_mean = (count1 * mean1 + count2 * mean2) / combined_count
    combined_M2 = M21 + M22 + delta ** 2 * count1 * count2 / combined_count

    # Compute variance only if there are enough samples
    combined_variance = combined_M2 / combined_count if combined_count > 1 else 0.0
    combined_meta = {"mean": combined_mean, "count": combined_count, "M2": combined_M2}

    return combined_variance, combined_meta


In [59]:
def aggregate_variances(arrays):
    """
    Compute aggregated variance for a list of arrays.
    
    Parameters:
        arrays (list of numpy.ndarray): List of input arrays.
    
    Returns:
        float: Aggregated variance across all arrays.
    """
    combined_meta = {"mean": 0.0, "count": 0, "M2": 0.0}

    for arr in arrays:
        var, meta = single_pass_variance(arr)
        print(var)
        var, combined_meta = combine_variance2(combined_meta, meta)
        

    aggregated_variance = combined_meta["M2"] / combined_meta["count"] if combined_meta["count"] > 1 else 0.0
    return aggregated_variance, combined_meta

In [57]:
def remove_variances(combined_meta, arrays):
    """
    Compute aggregated variance for a list of arrays.
    
    Parameters:
        arrays (list of numpy.ndarray): List of input arrays.
    
    Returns:
        float: Aggregated variance across all arrays.
    """
    for arr in arrays:
        var, meta = single_pass_variance(arr)
        print(var) 
        meta = {"mean": meta["mean"], "count": meta["count"]*-1, "M2": meta["M2"]}
        var, combined_meta = combine_variance2(combined_meta, meta)
        #print(var)
        

    aggregated_variance = combined_meta["M2"] / combined_meta["count"] if combined_meta["count"] > 1 else 0.0
    return aggregated_variance

In [28]:
def create_random_arrays_with_stats(configs):
    arrays = []

    for mean, var, size in configs:
        array = np.random.normal(mean, var, size)  # Generate an array of random numbers in [0, 1)
        arrays.append(array)
    return arrays

In [51]:
arrays = create_random_arrays_with_stats([(3, 2, 100), (2, 4, 100), (6, 5, 100), (10, 1, 100),  (10, 1, 100),
                                          (8, 0.5, 100), (3, 10, 100), (20, 5, 100), (10, 1, 100), (10, 0.5, 100),])

In [60]:
var, meta = aggregate_variances(arrays)
var

3.9490985200336097
11.931223298044271
25.73078695272483
1.0851983423957463
0.992166696309802
0.29179189548427603
120.38572704450824
18.59847634062613
1.0568454066942925
0.19658065004909278


np.float64(43.952082070543526)

In [62]:
merged = np.zeros(sum([len(arr) for arr in arrays]))
index = 0
for arr in arrays:
    print(arr.var(), arr.mean())
    end_idx = index + len(arr)
    merged[index:end_idx] = arr
    index = end_idx
print(f"Aggregated Variance: {merged.var()}")

3.94909852003361 2.7332805884529408
11.931223298044278 2.004909375806807
25.73078695272482 5.5702351731015
1.0851983423957463 9.973740887127002
0.9921666963098019 10.093332092121223
0.29179189548427603 7.969016493224489
120.38572704450817 3.9751851312637068
18.59847634062613 20.40008587671473
1.056845406694292 10.027983205534388
0.19658065004909267 10.035070029743183
Aggregated Variance: 43.952082070543526


In [54]:
remove_variances(meta, arrays)

45.47850194330586
45.97994955977115
53.36440242959675
62.41129552067168
75.06687969784686
91.93391843553991
140.08508680462984
183.59119467426794
368.2392096436914
0.0


0.0

In [55]:
for arr in arrays:
    n = len(arr)
    merged = merged[n:]
    if len(merged) > 0:
        print(merged.var(), merged.mean())
    else:
        print("END")

44.600924494409504
42.00986910525167
41.4755142093674
48.17919314960552
57.591490174043535
69.9437855830436
30.507758304962614
0.626725584141034
0.19658065004909267
END
